In [6]:
import numpy as np
import pandas as pd
import xgboost as xgb
from flair.data import Sentence

2022-10-26 09:44:45.077515: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
data = pd.read_csv("./train.csv")
data.head()

In [ ]:
data[data.is_duplicate == 1]

In [ ]:
data.groupby(["name_1"]).count()

In [ ]:
import spacy

NER = spacy.load("en_core_web_sm")

NER вещь интерсеная, но использовать его в одиночку для текущей задачи - бесполезно. Он выявляет сущности в тексте (организации, географические локации, имена, даты). Можно дополнительно к эмбедингам проверять является ли все текущее название названием организации.

In [ ]:
NER(data.iloc[1].name_2).ents, NER(data.iloc[1].name_1).ents

In [ ]:
for elem in data.iloc[1562].name_2.split():
    for word in NER(elem).ents:
        print(word.text, word.label_)

In [ ]:
for word in NER(data.iloc[0].name_1).ents:
    print(word.text, word.label_)

 Можно попробовать разные виды эмбедингов, а поверх бинарный класстфикатор

In [ ]:
from flair.embeddings import FlairEmbeddings

In [ ]:
# character_embeddings = CharacterEmbeddings()
flair_forward = FlairEmbeddings("news-forward-fast")

In [ ]:
sentence = Sentence(data.iloc[0].name_1)
# embed words in sentence
flair_forward.embed(sentence)
for token in sentence:
    print(token.embedding)
z = token.embedding.size()[0]

In [ ]:
import torch
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
data = data[:3000]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data[["name_1", "name_2"]], data.is_duplicate, test_size=0.33, random_state=42
)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
def get_embed(X):
    X_embed = []
    s = torch.zeros(0, z)
    # iterating Sentence (tqdm tracks progress) #
    for i in tqdm(range(data.shape[0])):
        tweet = data.iloc[i].name_1
        # empty tensor for words #
        w = torch.zeros(0, z)
        sentence = Sentence(tweet)

        flair_forward.embed(sentence)
        for token in sentence:
            # storing word Embeddings of each word in a sentence #
            w = torch.cat((w, token.embedding.view(-1, z)), 0)
            # storing sentence Embeddings (mean of embeddings of all words)   #
            s = torch.cat((s, w.mean(dim=0).view(-1, z)), 0)
        X_embed.append(s)
    return X_embed

In [16]:
# у меня не хватает мощностей опробовать хоть какие-то эмбединги на этой выборке
# можно на меньшей
X_train_embed = get_embed(X_train)

100%|███████████████████████████████████████| 3000/3000 [06:14<00:00,  8.00it/s]


In [ ]:
X_test_embed = get_embed(X_test)

 92%|███████████████████████████████████▊   | 2753/3000 [05:26<00:59,  4.14it/s]

In [1]:
model = xgb.XGBClassifier()
model.fit(np.array(X_train_embed), y_train)

In [ ]:
y_pred = model.predict(X_test_embed)
predictions = [round(value) for value in y_pred]

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(y_test, y_pred)